In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
CKPT_DIR = 'new_ckpt_dir_2'

In [3]:
class Network2(object):
    
    def __init__(self):
        self.global_step = tf.Variable(0,trainable = False)
        self.XX = tf.placeholder(tf.float32, [None, 784], name = "x_input")
        self.label = tf.placeholder(tf.float32, [None, 10], name = "y_input")
        self.W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1))
        self.B1 = tf.Variable(tf.zeros([200]))
        self.W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1))
        self.B2 = tf.Variable(tf.zeros([100]))
        self.W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1))
        self.B3 = tf.Variable(tf.zeros([60]))
        self.W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1))
        self.B4 = tf.Variable(tf.zeros([30]))
        self.W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1))
        self.B5 = tf.Variable(tf.zeros([10]))
        self.Y1 = tf.nn.sigmoid(tf.matmul(self.XX, self.W1) + self.B1)
        self.Y2 = tf.nn.sigmoid(tf.matmul(self.Y1, self.W2) + self.B2)
        self.Y3 = tf.nn.sigmoid(tf.matmul(self.Y2, self.W3) + self.B3)
        self.Y4 = tf.nn.sigmoid(tf.matmul(self.Y3, self.W4) + self.B4)
        self.Ylogits = tf.matmul(self.Y4, self.W5) + self.B5
        self.Y = tf.nn.softmax(self.Ylogits, name = "z_output")
        self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = self.Ylogits, labels = self.label)
        self.loss = tf.reduce_mean(self.cross_entropy)*100
        self.predict = tf.equal(tf.argmax(self.Y, 1), tf.argmax(self.label, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.predict, tf.float32))
        self.train = tf.train.AdamOptimizer(0.003).minimize(self.loss, global_step = self.global_step)

In [4]:
class Train(object):
    
    def __init__(self):
        self.net = Network2()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.data = input_data.read_data_sets('./dataset',one_hot = True)
        
    def train(self):
        batch_size = 64
        train_step = 10000
        step = 0
        #Save Model every 1000 steps
        save_interval = 1000
        saver = tf.train.Saver(max_to_keep = 10)
        ckpt = tf.train.get_checkpoint_state(CKPT_DIR)
        if ckpt and ckpt.get_checkpoint_state(CKPT_DIR):
            saver.restore(self.sess,ckpt.model_checkpoint_path)
            step = self.sess.run(self.net.global_step)
            print('continue from')
            print('  -> Minibatch update : ',step)
        while step < train_step:
            x,label = self.data.train.next_batch(batch_size)
            _,loss = self.sess.run([self.net.train,self.net.loss], feed_dict = {self.net.XX: x,self.net.label:label})
            step = self.sess.run(self.net.global_step)
            if step % 1000 == 0:
                print('No %6d step, current loss: %.3f'%(step,loss))
            if step % save_interval == 0:
                saver.save(self.sess,CKPT_DIR + '/model',global_step = step)
    
    def calculate_accuracy(self):
        test_x = self.data.test.images
        test_label = self.data.test.labels
        acc = self.sess.run(self.net.accuracy,feed_dict = {self.net.XX:test_x,self.net.label:test_label})
        print("acc: %.3f, Number of pic test: %d " % (acc, len(test_label)))

In [5]:
if __name__ == '__main__':
    model = Train()
    model.train()
    model.calculate_accuracy()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./dataset/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./dataset/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./dataset/t10k-images-idx3-ubyte.gz
Successfu